<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.metrics import (
    f1_score, 
    roc_auc_score, 
    roc_curve, 
    roc_auc_score,
    classification_report,
    recall_score
)
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

## Подготовка данных

In [2]:
data = pd.read_csv('C:/Users/emmik/Downloads/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


В первую очередь удалим колонки с индексом строки, уникальным идентификатором и фамилией клиента для того, чтобы качество модели показало наилучший результат, затем приведем наименования столбцов в нормальный вид.

In [3]:
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname']) 

In [4]:
data.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', i).lower() for i in data.columns]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   credit_score      10000 non-null  int64  
 1   geography         10000 non-null  object 
 2   gender            10000 non-null  object 
 3   age               10000 non-null  int64  
 4   tenure            10000 non-null  int64  
 5   balance           10000 non-null  float64
 6   num_of_products   10000 non-null  int64  
 7   has_cr_card       10000 non-null  int64  
 8   is_active_member  10000 non-null  int64  
 9   estimated_salary  10000 non-null  float64
 10  exited            10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [6]:
for colname in ['geography', 'gender']:
    print(data[colname].value_counts())

France     5014
Germany    2509
Spain      2477
Name: geography, dtype: int64
Male      5457
Female    4543
Name: gender, dtype: int64


In [7]:
data['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Объектов "нулевого" класса гораздо больше, чем "первого", что можно сказать о их дисбалансе, с которым  дальнейшем мы будем работать. 
Применим прямое кодирование ко всему датафрейму для отображения новых признаков (пол и страна).

In [8]:
data.head(5)

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
data.describe(include='all')

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
count,10000.000000,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
unique,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,France,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,5014,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,650.528800,NaN,NaN,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,NaN,NaN,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,NaN,NaN,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,NaN,NaN,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,NaN,NaN,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,NaN,NaN,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000


На первый взгляд, общая картина адекватная, но есть признаки с большими суммами, из-за чего они будут перетягивать на себя все внимание модели, необходимо данные масштабирвать.

В данном пункте мы подготовили данные, удалили ненужные колонки для машинного обучения, привели названия столбцов в нормальный вид, в данных присутствует дисбаланс классов. 

## Исследование задачи

Разобьем данные на обычный и целевой признаки, разобьем данные на тренировочную, валидационную и тестовую выборки в соотношении 60/20/20.

In [10]:
features = data.drop(columns='exited')
target = data['exited']

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, shuffle=True, random_state=42)
features_train, features_val, target_train, target_val = train_test_split(
    features_train, target_train, test_size=0.25, shuffle=True, random_state=42)

In [12]:
print(features_train.shape[0], features_test.shape[0], features_val.shape[0])

6000 2000 2000


Применим метод OneHotEncoder к категориальным признакам. 

In [13]:
categorial_signs = ['geography', 'gender']

In [14]:
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [15]:
encoder_ohe.fit(features_train[categorial_signs])

OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [16]:
features_train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_train[categorial_signs])

In [17]:
features_train = features_train.drop(categorial_signs, axis=1)

In [18]:
features_train.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
8588,712,29,7,77919.78,1,1,0,122547.58,0.0,1.0,0.0
3178,542,39,4,109949.39,2,1,1,41268.65,0.0,0.0,0.0
5200,512,42,9,93955.83,2,1,0,14828.54,1.0,0.0,1.0
8889,580,39,9,128362.59,1,1,0,86044.98,0.0,0.0,0.0
5789,689,55,1,76296.81,1,1,0,42364.75,1.0,0.0,0.0


In [19]:
features_val[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_val[categorial_signs])

In [20]:
features_val = features_val.drop(categorial_signs, axis=1)

In [21]:
features_val.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
3188,801,32,4,75170.54,1,1,1,37898.50,0.0,0.0,0.0
8293,766,28,4,90696.78,1,0,1,21597.20,1.0,0.0,0.0
1710,828,28,8,134766.85,1,1,0,79355.87,0.0,1.0,1.0
7510,798,36,1,0.00,2,1,1,159044.10,0.0,0.0,1.0
1461,611,34,4,0.00,2,1,0,170950.58,0.0,1.0,1.0


In [22]:
features_test[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_test[categorial_signs])

In [23]:
features_test = features_test.drop(categorial_signs, axis=1)

In [24]:
features_test.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
6252,596,32,3,96709.07,2,0,0,41788.37,1.0,0.0,1.0
4684,623,43,1,0.00,2,1,1,146379.30,0.0,0.0,1.0
1731,601,44,4,0.00,2,1,0,58561.31,0.0,1.0,0.0
4742,506,59,8,119152.10,2,1,1,170679.74,1.0,0.0,1.0
4521,560,27,7,124995.98,1,1,1,114669.79,0.0,1.0,0.0


Масштабируем наши данные. 

In [25]:
numeric  = ['credit_score', 'age', 'tenure', 'balance', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_val[numeric] = scaler.transform(features_val[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Далее, обучим наши модели на тренировочной выборке. 

In [26]:
%%time

f1_dtc = 0

for depth in range(1, 30, 2):
    model = DecisionTreeClassifier(random_state=41, max_depth=depth, min_samples_leaf=17)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_val)
    
    if f1_score(target_val, predicted_valid) > f1_dtc:
        f1_dtc = f1_score(target_val, predicted_valid)
    
print(f'F1-мера: {f1_dtc} \nAUC-ROC: {roc_auc_score(target_val, predicted_valid)}')

F1-мера: 0.5760709010339734 
AUC-ROC: 0.7029466107034913
Wall time: 380 ms


In [27]:
%%time

lr = LogisticRegression(random_state=42, solver='liblinear')
lr.fit(features_train, target_train)
predicted_valid = lr.predict(features_val)

print(f'F1-мера: {f1_score(target_val, predicted_valid)} \nAUC-ROC: {roc_auc_score(target_val, predicted_valid)}')

F1-мера: 0.3115942028985507 
AUC-ROC: 0.5874180501074865
Wall time: 31 ms


In [28]:
%%time

f1_rfc = 0

for est in range(10, 101, 10):
    for depth in range(1, 14, 2):
        model = RandomForestClassifier(random_state=42, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_val)
        
        if f1_score(target_val, predicted_valid) > f1_rfc:
            f1_rfc = f1_score(target_val, predicted_valid)
    
print(f'F1-мера: {f1_rfc} \nAUC-ROC: {roc_auc_score(target_val, predicted_valid)}')

F1-мера: 0.605143721633888 
AUC-ROC: 0.7200094289101371
Wall time: 18.9 s


На данном этапе мы разделили выборку на обычные и целевой признаки, разбили данные на тренировочную, валидационную и тестовую выборки в соотношении 60/20/20 и отмасштабировали данные. 

Далее, обучили модели на тренировочной выборке, лучшие показатели дала модель "Случайный лес" с показателями: 
F1-мера: 0.60
AUC-ROC: 0.72

## Борьба с дисбалансом

Как было сказано выше, в наших данных присутствовал дисбаланс классов. Для борьбы с ними будем использовать увеличение и уменьшение выборки. 

In [29]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=42)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=42)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=42)
    
    return features_downsampled, target_downsampled

In [30]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [31]:
%%time

f1_rfc = 0

for est in range(10, 101, 10):
    for depth in range(1, 14, 2):
        model = RandomForestClassifier(random_state=42, n_estimators=est, max_depth=depth)
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_val)
        
        if f1_score(target_val, predicted_valid) > f1_rfc:
            f1_rfc = f1_score(target_val, predicted_valid)
    
print(f'F1-мера: {f1_rfc} \nAUC-ROC: {roc_auc_score(target_val, predicted_valid)}')

F1-мера: 0.6032660902977907 
AUC-ROC: 0.7717646861378238
Wall time: 11.7 s


In [32]:
oversample = SMOTE(random_state=42)

In [33]:
features_train_up, target_train_up = oversample.fit_resample(features_train, target_train)

In [34]:
%%time

f1_rfc = 0

for est in range(10, 101, 10):
    for depth in range(1, 14, 2):
        model = RandomForestClassifier(random_state=42, n_estimators=est, max_depth=depth)
        model.fit(features_train_up, target_train_up)
        predicted_valid = model.predict(features_val)
        
        if f1_score(target_val, predicted_valid) > f1_rfc:
            f1_rfc = f1_score(target_val, predicted_valid)
    
print(f'F1-мера: {f1_rfc} \nAUC-ROC: {roc_auc_score(target_val, predicted_valid)}')

F1-мера: 0.6207627118644068 
AUC-ROC: 0.7561688415714239
Wall time: 29.8 s


In [35]:
def roc_auc(model, features, target):
    probabilities = model.predict_proba(features)
    probabilities_one = probabilities[:, 1]
    return roc_auc_score(target, probabilities_one)

In [36]:
f1_rfc = 0

for est in range(10, 101, 10):
    for depth in range(1, 14, 2):
        model = RandomForestClassifier(random_state=42, n_estimators=est, max_depth=depth)
        model.fit(features_train_up, target_train_up)
        predicted_valid = model.predict(features_val)
        
        if f1_score(target_val, predicted_valid) > f1_rfc:
            f1_rfc = f1_score(target_val, predicted_valid)
        
print(f'Лучшие параметры: n_estimators: {est}, max_depth: {depth}')
    
print(f'F1-мера: {f1_rfc} \nAUC-ROC: {roc_auc(model, features_val, target_val)}')

Лучшие параметры: n_estimators: 100, max_depth: 13
F1-мера: 0.6207627118644068 
AUC-ROC: 0.8637003742575496


На данном этапе мы привели класс к балансу двумя методами: увеличение и уменьшение выборки. Наиболее лучшим методом оказался метод upsample (увеличение выборки). При таком балансе и выборе наилучшей модели результаты оказались удовлетворительными: 

F1-мера: 0.62

AUC-ROC: 0.86

Лучшие параметры для случайного леса при n_estimators: 100 и max_depth: 13.

## Тестирование модели

In [37]:
rfc = RandomForestClassifier(random_state=14, n_estimators=105, max_depth=15)
rfc.fit(features_train_up, target_train_up)
predicted_test = rfc.predict(features_test)
        
print("F1-мера:", f1_score(target_test, predicted_test))
print("AUC-ROC:", roc_auc_score(target_test, predicted_test))

F1-мера: 0.6084788029925187
AUC-ROC: 0.7590946732726257


In [38]:
recall = recall_score(rfc.predict(features_test), target_test)
print(f'Recall = {recall}')

Recall = 0.5965770171149144


ВЫВОД:

В данной проектной работе были подготовлены данные для машинного обучения, масштабированы данные, проведено обучение на трех моделей на тренировочной выборке с дисбалансом классов, где самой лучшей моделью оказалась модель случайного леса, показавшая наилучшие результаты метрик.

Эта модель была использована на обучении с балансированным классом, где баланс классов достигался с помощью увеличения и уменьшения выборки, что немного повысило результат метрик. На тестовой выборке результат F1-мера: 0.61 и AUC-ROC: 0.76 c параметрами при random_state=14, n_estimators=105 и max_depth=15.

Recall модели составил 0.59, что позволило выявить 59% потенциально уходящих клиентов.